<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c22081be4af6634d8e4bb00c5e660f6bc4ad618c5ecd56b5e855e96c1cfcec0b
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-24 09:42:11
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.40 C
-------------------
Today PnL: -40.08 K (-0.29%)
Current PnL: -19.38 L (-13.04%)
CY Booked + Current PnL: -7.94 L (-5.34%)
-------------------
Total profit:  1.38 L
Total loss:  -20.76 L
-------------------
Total Booked + Current PnL: 19.04 L (15.64%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.16%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 85.25 L (60.8%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,11.93,55.0,H-MC,3.75,193170.0,28500.0,14835.0,-1.64,17.31,7.68,26.31,89.0,1.92,1.38,26.67,X40N,NTT,AC
77,TTKPRESTIG,770.00,96.16,39.0,M-SC,2.92,84534.0,-16243.0,16332.0,-0.62,-16.12,19.32,0.09,245.0,-0.99,0.60,11.06,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,59.0,H-MC,9.88,110304.0,9800.0,19303.0,-0.34,9.75,17.50,28.96,79.0,0.51,0.79,47.03,MH,ATH,METALS
57,RELIANCE,1533.00,-13.66,46.0,H-LC,4.45,216138.0,5772.0,23019.0,-0.31,2.74,10.65,13.68,37.0,0.25,1.54,19.72,XY25,NTT,REFINERIES
42,ITC,452.00,-39.19,40.0,H-LC,2.07,196676.0,-3462.0,24348.0,-0.25,-1.73,12.38,10.44,4.0,-0.14,1.40,3.74,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3840.54,59.0,M-SC,5.35,122862.0,-24506.0,76322.0,3.72,-16.63,62.12,35.16,236.0,-0.32,0.88,25.49,XY24,NTT,AUTO
69,SURYODAY,240.0,51.32,65.0,H-SC,1.07,138485.0,-40586.0,106315.0,1.65,-22.66,76.77,36.71,135.0,-0.38,0.99,37.41,XR,NTT,BANKS
21,COFFEEDAY,80.0,-41.65,54.0,L-SC,42.86,86784.0,-26765.0,66815.0,1.55,-23.57,76.99,35.27,268.0,-0.40,0.62,112.41,XR,NTT,HOTELS
61,SAMMAANCAP,326.0,-171.59,56.0,M-SC,4.90,82134.0,-20076.0,113468.0,1.38,-19.64,138.15,91.37,208.0,-0.18,0.59,33.33,XY25,NTT,FINANCE
0,5PAISA,593.0,124.76,48.0,H-SC,7.25,152298.0,-66198.0,94394.0,1.34,-30.30,61.98,12.90,153.0,-0.70,1.09,15.93,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,11.93,55.0,H-MC,3.75,193170.0,28500.0,14835.0,-1.64,17.31,7.68,26.31,89.0,1.92,1.38,26.67,X40N,NTT,AC
37,INDIAMART,4850.92,-55.39,25.0,H-SC,10.69,124656.0,1320.0,132447.0,-1.63,1.07,106.25,108.45,119.0,0.01,0.89,23.66,AR,ATH,MISC
16,CAMPUS,393.00,-25.73,49.0,M-SC,0.42,154796.0,-12979.0,68822.0,-1.57,-7.74,44.46,33.28,210.0,-0.19,1.10,19.98,XY24,NTT,FOOTWEAR
73,TATAMOTORS,1065.00,-20.84,47.0,M-LC,2.35,250633.0,-25890.0,135968.0,-1.55,-9.36,54.25,39.81,54.0,-0.19,1.79,20.12,XY24,NTT,AUTO
29,HAVELLS,2069.16,-2.49,45.0,H-MC,2.11,241706.0,-6025.0,81092.0,-1.46,-2.43,33.55,30.30,92.0,-0.07,1.72,10.54,X40,ATH,ELECTRICAL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.57,42.0,H-MC,2.02,134956.0,-832.0,43591.0,-0.62,-0.61,32.30,31.48,107.0,-0.02,0.96,11.43,X40,ATH,INSURANCE
23,DABUR,735.00,-7.98,38.0,H-MC,6.80,248999.0,931.0,107468.0,-0.31,0.38,43.16,43.70,102.0,0.01,1.78,14.88,XY24,BTT,FMCG
52,PGHH,17973.08,-30.28,53.0,H-MC,3.67,201765.0,945.0,67833.0,0.32,0.47,33.62,34.25,80.0,0.01,1.44,5.69,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.61,121635.0,-20844.0,40371.0,-0.35,-14.63,33.19,13.70,163.0,-0.52,0.87,43.56,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,96.16,39.0,M-SC,2.92,84534.0,-16243.0,16332.0,-0.62,-16.12,19.32,0.09,245.0,-0.99,0.60,11.06,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,52.0,H-SC,1.12,221724.0,-47943.0,84277.0,0.06,-17.78,38.01,13.47,138.0,-0.57,1.58,12.14,XY24,NTT,PAINTS
18,CERA,9475.0,-23.15,33.0,H-SC,2.16,140300.0,-35603.0,77628.0,-1.29,-20.24,55.33,23.89,149.0,-0.46,1.00,21.17,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-22.53,42.0,H-MC,6.90,104899.0,-26936.0,69160.0,-0.62,-20.43,65.93,32.03,98.0,-0.39,0.75,17.50,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1249.28,78.0,M-SC,10.01,183548.0,2405.0,43060.0,-0.74,1.33,23.46,25.10,235.0,0.06,1.31,47.10,XY24,NTT,HEALTHCARE
60,SAIL,228.00,50.76,72.0,M-MC,9.80,237808.0,12846.0,154813.0,0.58,5.71,65.10,74.53,192.0,0.08,1.70,39.81,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,11.93,55.0,H-MC,3.75,193170.0,28500.0,14835.0,-1.64,17.31,7.68,26.31,89.0,1.92,1.38,26.67,X40N,NTT,AC
17,CAMS,5211.76,-2.73,54.0,H-SC,2.22,110709.0,8705.0,35217.0,-0.81,8.53,31.81,43.06,122.0,0.25,0.79,27.11,X40N,ATH,MISC
84,VOLTAS,1530.00,-3.04,42.0,H-MC,2.94,203490.0,11748.0,26006.0,-1.17,6.13,12.78,19.69,99.0,0.45,1.45,13.34,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,59.0,H-MC,9.88,110304.0,9800.0,19303.0,-0.34,9.75,17.50,28.96,79.0,0.51,0.79,47.03,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.17,48.0,M-SC,1.52,99435.0,7936.0,70818.0,0.37,8.67,71.22,86.07,223.0,0.11,0.71,44.40,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,69.0,H-MC,7.45,187282.0,7474.0,104934.0,0.28,4.16,56.03,62.52,88.0,0.07,1.34,39.66,XR,NTT,BANKS
86,WIPRO,420.00,-12.89,43.0,M-LC,5.37,152706.0,1761.0,107688.0,-1.33,1.17,70.52,72.51,53.0,0.02,1.09,7.26,XR,NTT,IT
37,INDIAMART,4850.92,-55.39,25.0,H-SC,10.69,124656.0,1320.0,132447.0,-1.63,1.07,106.25,108.45,119.0,0.01,0.89,23.66,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-19.44,34.0,H-LC,7.70,290354.0,-25488.0,138150.0,-0.74,-8.07,47.58,35.67,5.0,-0.18,2.07,4.67,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.94,38.0,H-LC,9.97,223458.0,-18466.0,81741.0,-0.45,-7.63,36.58,26.15,8.0,-0.23,1.59,5.60,X40,ATH,IT
31,HINDUNILVR,2922.00,-10.18,38.0,H-LC,9.92,252310.0,-4753.0,39890.0,0.04,-1.85,15.81,13.67,24.0,-0.12,1.80,18.18,XY25,NTT,FMCG
42,ITC,452.00,-39.19,40.0,H-LC,2.07,196676.0,-3462.0,24348.0,-0.25,-1.73,12.38,10.44,4.0,-0.14,1.40,3.74,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-14.50,40.0,H-LC,7.54,215741.0,-36027.0,89230.0,0.19,-14.31,41.36,21.13,27.0,-0.40,1.54,16.05,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.20,47.0,H-LC,1.54,234945.0,-11206.0,90407.0,-0.79,-4.55,38.48,32.18,16.0,-0.12,1.68,29.85,X200,ATH,IT
42,ITC,452.0,-39.19,40.0,H-LC,2.07,196676.0,-3462.0,24348.0,-0.25,-1.73,12.38,10.44,4.0,-0.14,1.40,3.74,X40,NTT,FMCG
65,SIEMENS,4671.5,0.17,52.0,H-LC,3.07,160990.0,-25105.0,72590.0,-0.64,-13.49,45.09,25.51,15.0,-0.35,1.15,18.64,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.66,46.0,H-LC,4.45,216138.0,5772.0,23019.0,-0.31,2.74,10.65,13.68,37.0,0.25,1.54,19.72,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.53,42.0,H-LC,4.46,272212.0,6786.0,50005.0,-0.44,2.56,18.37,21.40,11.0,0.14,1.94,9.83,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.36,41.0,H-LC,12.12,286089.0,-59867.0,134719.0,-0.62,-17.30,47.09,21.64,1.0,-0.44,2.04,1.35,X40,ATH,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.07,196676.0,-3462.0,24348.0,-0.25,-1.73,12.38,10.44,4.0,-0.14,1.40,3.74,X40,NTT,FMCG
83,VBL,671.64,-19.44,34.0,H-LC,7.70,290354.0,-25488.0,138150.0,-0.74,-8.07,47.58,35.67,5.0,-0.18,2.07,4.67,X40N,ATH,FMCG
80,UNITDSPR,1644.00,-11.95,56.0,H-LC,7.29,230772.0,-3372.0,51993.0,0.04,-1.44,22.53,20.77,86.0,-0.06,1.65,5.38,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.94,38.0,H-LC,9.97,223458.0,-18466.0,81741.0,-0.45,-7.63,36.58,26.15,8.0,-0.23,1.59,5.60,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.17,52.0,H-LC,3.07,160990.0,-25105.0,72590.0,-0.64,-13.49,45.09,25.51,15.0,-0.35,1.15,18.64,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,40.0,H-LC,2.07,196676.0,-3462.0,24348.0,-0.25,-1.73,12.38,10.44,4.0,-0.14,1.40,3.74,X40,NTT,FMCG
19,CIPLA,1795.00,-21.84,41.0,H-LC,5.12,208706.0,4206.0,37212.0,-0.24,2.06,17.83,20.25,10.0,0.11,1.49,10.97,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-14.50,40.0,H-LC,7.54,215741.0,-36027.0,89230.0,0.19,-14.31,41.36,21.13,27.0,-0.40,1.54,16.05,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,46.0,H-LC,4.45,216138.0,5772.0,23019.0,-0.31,2.74,10.65,13.68,37.0,0.25,1.54,19.72,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.36,41.0,H-LC,12.12,286089.0,-59867.0,134719.0,-0.62,-17.30,47.09,21.64,1.0,-0.44,2.04,1.35,X40,ATH,IT
40,INFY,2275.00,-18.52,48.0,H-LC,8.12,317945.0,5057.0,166635.0,-0.32,1.62,52.41,54.87,3.0,0.03,2.27,8.37,X40,BTT,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.07,196676.0,-3462.0,24348.0,-0.25,-1.73,12.38,10.44,4.0,-0.14,1.40,3.74,X40,NTT,FMCG
83,VBL,671.64,-19.44,34.0,H-LC,7.70,290354.0,-25488.0,138150.0,-0.74,-8.07,47.58,35.67,5.0,-0.18,2.07,4.67,X40N,ATH,FMCG
1,ABB,7934.00,-38.99,54.0,H-LC,8.61,254832.0,-6787.0,125989.0,0.19,-2.59,49.44,45.57,7.0,-0.05,1.82,8.43,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6966.67,60.0,L-SC,5.41,77514.0,-16296.0,92916.0,0.24,-17.37,119.87,81.67,269.0,-0.18,0.55,50.11,XR,NTT,CERAMICS
61,SAMMAANCAP,326.00,-171.59,56.0,M-SC,4.90,82134.0,-20076.0,113468.0,1.38,-19.64,138.15,91.37,208.0,-0.18,0.59,33.33,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,54.0,L-SC,42.86,86784.0,-26765.0,66815.0,1.55,-23.57,76.99,35.27,268.0,-0.40,0.62,112.41,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.69,41.0,H-SC,11.75,86843.0,-14120.0,108649.0,-0.24,-13.99,125.11,93.63,148.0,-0.13,0.62,29.35,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,50.0,H-SC,12.67,94080.0,-3900.0,25505.0,-0.25,-3.98,27.11,22.05,152.0,-0.15,0.67,36.75,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1249.28,78.0,M-SC,10.01,183548.0,2405.0,43060.0,-0.74,1.33,23.46,25.10,235.0,0.06,1.31,47.10,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-15.81,71.0,H-SC,15.38,225950.0,-44811.0,374761.0,0.26,-16.55,165.86,121.86,133.0,-0.12,1.61,76.76,AR,ATH,IT
7,ATULAUTO,844.00,3840.54,59.0,M-SC,5.35,122862.0,-24506.0,76322.0,3.72,-16.63,62.12,35.16,236.0,-0.32,0.88,25.49,XY24,NTT,AUTO
81,VAIBHAVGBL,521.00,59.78,60.0,H-SC,5.97,138614.0,-44161.0,155234.0,-0.56,-24.16,111.99,60.77,125.0,-0.28,0.99,24.15,XR,NTT,JEWELLERY
60,SAIL,228.00,50.76,72.0,M-MC,9.80,237808.0,12846.0,154813.0,0.58,5.71,65.10,74.53,192.0,0.08,1.70,39.81,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.18
1,25,43.64
2,50,74.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.13
LC    32.36
MC    23.52
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.39
X40      14.68
X40N     11.41
XR       11.33
XY25     10.01
AR        8.39
OX40N     7.83
X200      1.68
MH        1.64
X5K       1.42
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.47
H-LC    25.89
H-MC    20.54
M-SC    13.13
M-LC     5.43
M-MC     2.67
L-SC     1.53
L-LC     1.04
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.83,-5.86,40.79
IT,12.96,-15.81,78.96
FINANCE,10.12,-15.78,66.30
MISC,7.00,-16.99,82.03
BANKS,6.19,-15.18,75.77
PAINTS,5.63,-16.99,34.43
ELECTRICAL,5.48,-7.20,44.81
INSURANCE,3.72,-6.12,45.80
AC,3.53,2.12,14.27


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3121913.0,22
XR,1323512.0,14
AR,1154248.0,9
X40,789113.0,10
X40N,607361.0,9
OX40N,549315.0,10
XY25,436863.0,7
SR,249300.0,2
X5K,118159.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3666280.0,29
M-SC,1401925.0,17
H-MC,1228604.0,15
H-LC,1190778.0,15
M-LC,388919.0,4
M-MC,303516.0,2
L-SC,246074.0,3
L-MC,59706.0,1
L-LC,39046.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1226544.0      6
M-SC       XY24       799124.0      7
H-SC       AR         789758.0      5
           XR         775561.0      7
H-MC       XY24       564807.0      4
H-LC       X40        496673.0      5
M-MC       XY24       303516.0      2
H-SC       X40N       300886.0      4
           OX40N      258877.0      4
           SR         249300.0      2
H-LC       X40N       227355.0      3
H-MC       X40        219845.0      4
H-LC       AR         198579.0      2
H-MC       XY25       171435.0      2
M-SC       AR         165911.0      2
M-LC       XY24       163072.0      2
L-SC       XR         159731.0      2
M-SC       OX40N      134935.0      4
M-LC       X5K        118159.0      1
M-SC       XR         115892.0      2
           XY25       113468.0      1
H-LC       XY25       112914.0      3
M-LC       XR         107688.0      1
H-MC       XR         104934.0      1
H-LC       X200        90407.0      1
L-SC       OX40N       86343.0      1
H-MC       X40N        79120.0      2
M-SC       X40         72595.0      1
H-MC       OX40N       69160.0      1
H-SC       MH          65354.0      1
H-LC       XY24        64850.0      1
L-MC       XR          59706.0      1
L-LC       XY25        39046.0      1
H-MC       MH          19303.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
